In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import Counter
import datetime
import sys
import logging
import math
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.integrate import simps

In [ ]:
sensor_df = pd.DataFrame.from_csv("../../data/database/sensor_data.csv")
sensor_df = sensor_df[~((sensor_df.TagName == 'Start1') | (sensor_df.TagName == 'Start2'))]
Counter(sensor_df.TagName)

In [ ]:
tag_tester_groupby = sensor_df.groupby(['tester_id','TagName'])

In [ ]:
acceleration_data = pd.DataFrame(columns=['tester_id','TagName','Time','acc_val1','acc_val2','acc_val3',
                                          'acc_val4','gravity1','gravity2','gravity3','mag_val1',
                                          'mag_val2','mag_val3'])

In [ ]:
for key in tag_tester_groupby.groups.keys():
    cur_res = []
    prev_mag = None
    prev_grav = None

    for cur_sensor in tag_tester_groupby.get_group(key).values:
        if cur_sensor[1] == 9:
            prev_grav = cur_sensor[2:5].tolist()
        elif cur_sensor[1] == 2:
            prev_mag = cur_sensor[2:5].tolist()
        elif cur_sensor[1] == 10:
            if ((prev_grav is not None) and (prev_mag is not None)):
                cur_line = [key[0],key[1],cur_sensor[0]] + cur_sensor[2:5].tolist() + [0] + prev_grav + prev_mag
                cur_res.append(cur_line)
    acceleration_data = acceleration_data.append(pd.DataFrame(cur_res,columns=['tester_id','TagName','Time','acc_val1','acc_val2','acc_val3',
                                          'acc_val4','gravity1','gravity2','gravity3','mag_val1',
                                          'mag_val2','mag_val3']), ignore_index = True)
    

In [ ]:
acceleration_data.to_csv("../../data/linear_acceleration_grav_mag.csv")

In [ ]:
sensor_df[sensor_df.SENSORTYPE == 10].groupby(['tester_id', 'TagName']).count().head()

In [ ]:
acceleration_data.groupby(['tester_id', 'TagName']).groups.keys()

In [ ]:
acceleration_data.groupby(['tester_id', 'TagName']).count().head()

In [ ]:
linear_acc_df = pd.DataFrame(columns=col_names)
count = 0
for key in linear_acc_groups.groups.keys():
    cur_group = linear_acc_groups.get_group(key)
    t = cur_group.unix_timestamp.values
    v_1 = []
    d_1 = []
    v_2 = []
    d_2 = []
    v_3 = []
    d_3 = []
    for i in range(len(t)):
        v_1.append(simps(0.8*cur_group.VALUES1.values[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_1.append(simps(v_1[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        v_2.append(simps(0.8*cur_group.VALUES2.values[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_2.append(simps(v_2[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        v_3.append(simps(0.8*cur_group.VALUES3.values[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_3.append(simps(v_3[:i+1], x=t[:i+1]))
    
    v_1 = np.array(v_1)
    d_1 = np.array(d_1)
    v_2 = np.array(v_2)
    d_2 = np.array(d_2)
    v_3 = np.array(v_3)
    d_3 = np.array(d_3)

    temp = np.concatenate((t.reshape(-1,1),np.array([key[0]]*len(t)).reshape(-1,1),np.array([key[1]]*len(t)).reshape(-1,1),d_1.reshape(-1,1),d_2.reshape(-1,1),d_3.reshape(-1,1),v_1.reshape(-1,1),v_2.reshape(-1,1),v_3.reshape(-1,1)), axis = 1)

    linear_acc_df = linear_acc_df.append(pd.DataFrame(temp, columns=col_names), ignore_index=True)
    count+=1
    if (count%50==0):
        print(count)

    

In [ ]:
linear_acc_df.head()

In [ ]:
linear_acc_df.to_csv("../../data/linear_accuracy_features_df.csv")

In [ ]:
[0]*10